<a href="https://colab.research.google.com/github/mohitsaini65/Cotton-Plant-Disease-Prediction/blob/main/Cotton_Plant_Disease_Prediction_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32

train_path = "/content/drive/MyDrive/Colab Notebooks/Cotton-Plant-Disease-Dataset/data/train"
val_path   = "/content/drive/MyDrive/Colab Notebooks/Cotton-Plant-Disease-Dataset/data/val"
test_path  = "/content/drive/MyDrive/Colab Notebooks/Cotton-Plant-Disease-Dataset/data/test"

In [ ]:
resnet = ResNet50(
    input_shape=IMAGE_SIZE + [3],
    weights='imagenet',
    include_top=False
)

for layer in resnet.layers:
    layer.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
folders = glob(train_path + "/*")

x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,989,124 (91.51 MB)

 Trainable params: 401,412 (1.53 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_set = val_datagen.flow_from_directory(
    val_path,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.
Found 18 images belonging to 4 classes.


In [ ]:
history = model.fit(
    training_set,
    validation_data=val_set,
    epochs=20
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 59s 751ms/step - accuracy: 0.6506 - loss: 5.9897 - val_accuracy: 0.8981 - val_loss: 1.8888
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 34s 557ms/step - accuracy: 0.9272 - loss: 1.0233 - val_accuracy: 0.9259 - val_loss: 0.9348
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 544ms/step - accuracy: 0.9424 - loss: 0.6988 - val_accuracy: 0.9506 - val_loss: 0.9039
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 34s 559ms/step - accuracy: 0.9469 - loss: 0.7163 - val_accuracy: 0.9352 - val_loss: 1.1808
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 547ms/step - accuracy: 0.9565 - loss: 0.7114 - val_accuracy: 0.9599 - val_loss: 0.7172
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 34s 556ms/step - accuracy: 0.9518 - loss: 0.7850 - val_accuracy: 0.9691 - val_loss: 0.5140
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 547ms/step - accuracy: 0.9636 - loss: 0.6448 - val_accuracy: 0.9414 - val_loss: 0.7082
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 544ms/step - accuracy: 0.9648 - loss: 0.5516 - val_accu

In [ ]:
test_loss, test_acc = model.evaluate(test_set)
print("Test Accuracy:", test_acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.0


In [ ]:
model.save("/content/drive/MyDrive/model_resnet50_cotton.h5")

In [ ]:
img = image.load_img(
    "/content/drive/MyDrive/Colab Notebooks/Cotton-Plant-Disease-Dataset/data/train/diseased cotton leaf/dis_leaf (1)_iaip.jpg",
    target_size=(224,224)
)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
print("Prediction:", pred)
print("Class:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Prediction: [[1.000000e+00 0.000000e+00 1.456867e-39 0.000000e+00]]
Class: 0


In [ ]:
img = image.load_img(
    "/content/drive/MyDrive/Colab Notebooks/Cotton-Plant-Disease-Dataset/data/test/fresh cotton plant/dsd (223).jpg",
    target_size=(224,224)
)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
print("Prediction:", pred)
print("Class:", np.argmax(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Prediction: [[3.2537544e-30 0.0000000e+00 6.0101691e-42 1.0000000e+00]]
Class: 3
